In [ ]:
### 문제정의
- 네이버 영화리뷰 데이터셋을 이용해서 긍정/부정 분류기(모델)를 만들자.
- TF-IDF 방법을 적용해 보자
- konlpy 한국어 형태소 분석기를 설치하고 활용해 보자
- 단어별 긍정 / 부정 정보를 시각화해보자


In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.7 MB/s eta 0:00:00


In [5]:
from konlpy.tag import Okt

In [6]:
okt = Okt()

In [7]:
okt.morphs('아버지가방에들어가신다')
# morphs : 가지고 있는 문장을 명사단위로 쪼개주는 기능
# 한글은 띄어쓰기 단위로 토큰화를 진행하게 될 경우 의미가 손실
# 토큰화를 할 수 있는 모듈을 따로 사용해야 함

['아버지', '가방', '에', '들어가신다']

Konlpy 간단하게 사용해보기

In [16]:
text = ['우리반 오늘도 화이팅.',
        '우리반 내일도 화이팅.',
        '우리반 고생했어요.',
        '우리반 최고.']

Countvectorizer(BOW) : 토큰화

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
# 모델생성
CV = CountVectorizer()

In [17]:
# 학습
# 토큰화, 단어사전 구축
CV.fit(text)
# 토큰화 기본값 - 띄어쓰기 단위로 토큰화

CountVectorizer()

In [18]:
CV.vocabulary_

{'우리반': 3, '오늘도': 2, '화이팅': 5, '내일도': 1, '고생했어요': 0, '최고': 4}

Countvectorizer + Okt

In [20]:
# 함수 생성 (명사단위)
def myToken(text) :
    return okt.nouns(text)  # 명사단위로 추출해서 토큰화

In [22]:
# 모델 생성
cv_okt = CountVectorizer(tokenizer=myToken)

In [23]:
# 학습
cv_okt.fit(text)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function myToken at 0x78c42b192320>)

In [24]:
cv_okt.vocabulary_

{'우리': 4, '반': 2, '오늘': 3, '화이팅': 6, '내일': 1, '고생': 0, '최고': 5}

In [25]:
# 토큰화 도구 : morphs
# okt.morphs ==> 형태소별로 구분
okt.morphs(text[0])

['우리', '반', '오늘', '도', '화이팅', '.']

In [26]:
okt.pos(text[0])
# okt.pos : 토큰화 한 후 어떤 형태소를 가지고 있는지 확인해주는 함수

[('우리', 'Noun'),
 ('반', 'Noun'),
 ('오늘', 'Noun'),
 ('도', 'Josa'),
 ('화이팅', 'Noun'),
 ('.', 'Punctuation')]

In [27]:
# okt가 가지고 있는 전체 형태소 확인
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()
kkma.tagset
# 데이터 전처리시 시간적 여유가 많다면 사용
# okt에 비해 더 많은 형태소로 구분 ==> 시간이 많이걸림

네이버 영화리뷰 데이터셋 감성분석

In [31]:
import numpy as np
import pandas as pd

데이터 불러오기

In [33]:
text_train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings_train.csv', encoding='utf-8')
text_test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings_test.csv', encoding='utf-8')


In [ ]:
text_train

In [ ]:
text_test

In [35]:
text_test.info()
# document에 결측치 ==> 부정인지 긍정인지 평가만하고 문자평은 남기지 않은 데이터

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [36]:
# 결측치가 있는 행을 삭제
text_train.dropna(inplace=True)  #5
text_test.dropna(inplace=True)   #3

In [38]:
# 문제만 토큰화시키기 위해서 문제와 정답으로 나누기
X_train = text_train['document']
X_test = text_test['document']
y_train = text_train['label']
y_test = text_test['label']

In [39]:
# TF-IDF 적용
# 문장의 빈도(가중치, 패널티 적용)

In [40]:
 from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
# pipeline 으로 연결
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [42]:
pipe_model = make_pipeline(TfidfVectorizer(tokenizer=myToken),
                           LogisticRegression())

In [43]:
# 학습
pipe_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function myToken at 0x78c42b192320>)),
                ('logisticregression', LogisticRegression())])

In [45]:
# 평가
pipe_model.score(X_test, y_test)

0.768846130767846

In [46]:
# pipe_model --> TF-IDF(단어사전), LogisticRegression(가중치 - coef_)

tfidf = pipe_model.steps[0][1] # 단어사전
logi = pipe_model.steps[1][1] # 가중치
# steps : 단어사전에서 하나씩 꺼내오기

In [48]:
word_weights = logi.coef_[0] # 2차원에서 하나의 데이터를 꺼내오기
voca = tfidf.vocabulary_

In [ ]:
# 가중치를 기준으로 내림차순 정렬

# 단어들을 인덱스 순서대로 정렬
df = pd.DataFrame([voca.keys(), voca.values()])
# 번호를 인덱스로 사용하기 위해 전치, 행과 열을 뒤집음
df = df.T
# 1번 컬럼을 기준으로 정렬
df = df.sort_values(by=1)

# 가중치(coef)컬럼 추가
df['coef']= word_weights
# 가중치를 기준으로 내림차순 정렬
df = df.sort_values(by='coef',ascending=False)
df